In [ ]:
# @title 🖼️ Bing Image Downloader (NSFW Enabled)
# @markdown Enter your search preferences below and run the cell to download images.

# Install required library
!pip install icrawler --quiet

import os
import time
from icrawler.builtin import BingImageCrawler
from google.colab import files

# Form parameters
SEARCH_TERM = "Car"  # @param {type: "string"}
NUM_IMAGES = 10  # @param {type: "slider", min: 1, max: 200, step: 1}
BASE_DIR = "/content/downloaded_images"  # @param {type: "string"}
SHOW_PROGRESS = True  # @param {type: "boolean"}
CREATE_ZIP = False  # @param {type: "boolean"}

class NSFWBingCrawler(BingImageCrawler):
    def _request(self, url, **kwargs):
        if "adlt=" in url:
            url = url.replace("adlt=strict", "adlt=off")
        else:
            url += "&adlt=off"
        return super()._request(url, **kwargs)

def download_images(query, num_images, base="/content/downloaded_images"):
    """
    Download images from Bing with NSFW content allowed
    """
    # Create save directory
    save_dir = os.path.join(base, query.replace(" ", "_"))
    os.makedirs(save_dir, exist_ok=True)
    
    if SHOW_PROGRESS:
        print(f"🔍 Search term: '{query}'")
        print(f"📊 Number of images: {num_images}")
        print(f"📁 Save directory: {save_dir}")
        print("⏳ Starting download...")
    
    # Initialize and run crawler
    crawler = NSFWBingCrawler(storage={"root_dir": save_dir})
    crawler.crawl(keyword=query, max_num=num_images)
    
    return save_dir

def create_zip(folder_path):
    """Create a zip file of the downloaded images"""
    zip_path = folder_path + ".zip"
    !zip -r {zip_path} {folder_path}
    return zip_path

# Main execution
print("=" * 60)
print("BING IMAGE DOWNLOADER - COLAB FORM VERSION")
print("=" * 60)
print()

try:
    # Validate inputs
    if not SEARCH_TERM.strip():
        print("❌ Error: Search term cannot be empty!")
    else:
        # Download images
        if SHOW_PROGRESS:
            print(f"🚀 Downloading {NUM_IMAGES} images for: '{SEARCH_TERM}'")
        
        path = download_images(SEARCH_TERM, NUM_IMAGES, BASE_DIR)
        
        # Count downloaded files
        downloaded_files = [f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.webp'))]
        actual_count = len(downloaded_files)
        
        print()
        print("✅ Download completed!")
        print(f"📁 Location: {path}")
        print(f"📊 Images downloaded: {actual_count}/{NUM_IMAGES}")
        
        # Create zip if requested
        if CREATE_ZIP:
            print()
            print("📦 Creating zip file...")
            zip_file = create_zip(path)
            print(f"✅ Zip created: {zip_file}")
            
            # Offer download
            print()
            print("💾 Download zip file to your computer:")
            files.download(zip_file)
        
        # Show file listing
        if SHOW_PROGRESS and actual_count > 0:
            print()
            print("📋 Downloaded files:")
            for i, file in enumerate(downloaded_files[:10], 1):  # Show first 10 files
                print(f"  {i}. {file}")
            if actual_count > 10:
                print(f"  ... and {actual_count - 10} more files")

except Exception as e:
    print(f"❌ Error occurred: {str(e)}")
    print("Please check your inputs and try again.")

print()
print("=" * 60)